In [1]:
import os
import sys
import torch
import pandas as pd
from tqdm.notebook import tqdm
import pickle as pk
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

device = torch.cuda.set_device(0)

BASE_DIR = ".."
MODEL_BASE_DIR = f"{BASE_DIR}/best_models"
DATA_DIR = f"{BASE_DIR}/nbdata"
os.makedirs(DATA_DIR,exist_ok=True)
sys.path.append(BASE_DIR)

In [17]:
import torch
from torch import nn

In [2]:
from tdc import utils
names = utils.retrieve_benchmark_names('DTI_DG_Group')

In [3]:
names

['bindingdb_patent']

In [4]:
from tdc.benchmark_group import dti_dg_group
group = dti_dg_group(path = DATA_DIR)

Found local copy...


In [5]:
benchmark = group.get('bindingdb_patent')

In [6]:
predictions = {}
name = benchmark['name']
train_val, test = benchmark['train_val'], benchmark['test']

## Natural log transformed (kd/ki/ic50??)

In [7]:
all_drugs = pd.concat([train_val,test]).Drug.values
all_proteins = pd.concat([train_val,test]).Target.values

In [8]:
to_disk_path = f"{DATA_DIR}/tdc_bindingdb_patent_train"

In [9]:
from mol_feats import Morgan_f
from prot_feats import Prose_f

In [10]:
mol_featurizer = Morgan_f()
prot_featurizer = Prose_f()

In [11]:
mol_featurizer.precompute(all_drugs,to_disk_path=to_disk_path,from_disk=True)

  0%|                                                                                      | 138/232458 [00:00<02:49, 1374.55it/s]

--- precomputing morgan molecule featurizer ---


  3%|██▍                                                                                  | 6708/232458 [00:03<02:07, 1765.65it/s]RDKit ERROR: [12:54:18] Explicit valence for atom # 20 N, 4, is greater than permitted
[12:54:18] Explicit valence for atom # 20 N, 4, is greater than permitted
RDKit ERROR: [12:54:18] Explicit valence for atom # 20 N, 4, is greater than permitted
[12:54:18] Explicit valence for atom # 20 N, 4, is greater than permitted
  3%|██▌                                                                                  | 6892/232458 [00:03<02:09, 1737.33it/s]RDKit ERROR: [12:54:18] Explicit valence for atom # 4 N, 4, is greater than permitted
[12:54:18] Explicit valence for atom # 4 N, 4, is greater than permitted
RDKit ERROR: [12:54:18] Explicit valence for atom # 10 N, 4, is greater than permitted
[12:54:18] Explicit valence for atom # 10 N, 4, is greater than permitted
RDKit ERROR: [12:54:18] Explicit valence for atom # 10 N, 4, is greater than permitted
[12:54:18] 

rdkit not found this smiles for morgan: None convert to all 0 features
rdkit not found this smiles for morgan: None convert to all 0 features
rdkit not found this smiles for morgan: None convert to all 0 features
rdkit not found this smiles for morgan: None convert to all 0 features
rdkit not found this smiles for morgan: None convert to all 0 features
rdkit not found this smiles for morgan: None convert to all 0 features
rdkit not found this smiles for morgan: None convert to all 0 features
rdkit not found this smiles for morgan: None convert to all 0 features
rdkit not found this smiles for morgan: None convert to all 0 features


  3%|██▋                                                                                  | 7382/232458 [00:04<01:48, 2082.42it/s]RDKit ERROR: [12:54:18] Explicit valence for atom # 4 N, 4, is greater than permitted
[12:54:18] Explicit valence for atom # 4 N, 4, is greater than permitted
RDKit ERROR: [12:54:18] Explicit valence for atom # 19 N, 4, is greater than permitted
[12:54:18] Explicit valence for atom # 19 N, 4, is greater than permitted
  3%|██▉                                                                                  | 7993/232458 [00:04<01:35, 2362.78it/s]RDKit ERROR: [12:54:18] Can't kekulize mol.  Unkekulized atoms: 6 7 8 10 11 13 14
RDKit ERROR: 
[12:54:18] Can't kekulize mol.  Unkekulized atoms: 6 7 8 10 11 13 14



...

rdkit not found this smiles for morgan: None convert to all 0 features


 97%|████████████████████████████████████████████████████████████████████████████████▏  | 224444/232458 [01:36<00:04, 1797.25it/s]RDKit ERROR: [12:55:51] Explicit valence for atom # 1 N, 4, is greater than permitted
[12:55:51] Explicit valence for atom # 1 N, 4, is greater than permitted


rdkit not found this smiles for morgan: None convert to all 0 features


100%|███████████████████████████████████████████████████████████████████████████████████| 232458/232458 [01:37<00:00, 2384.31it/s]


--- saving morgans to ../nbdata/tdc_bindingdb_patent_train_Morgan_MOLECULES.pk ---


In [13]:
prot_featurizer.precompute(all_proteins,to_disk_path=to_disk_path,from_disk=True)

  0%|                                                                                        | 1/232458 [00:00<9:32:18,  6.77it/s]

--- precomputing Prose protein featurizer ---


100%|███████████████████████████████████████████████████████████████████████████████████| 232458/232458 [00:54<00:00, 4256.15it/s]

--- saving protein embeddings to ../nbdata/tdc_bindingdb_patent_train_Prose_f_PROTEINS.pk ---


In [ ]:
## --- train your model --- ##

In [82]:
class SimplePLMModel(nn.Module):
    def __init__(self,
                 mol_emb_size = 2048,
                 prot_emb_size = 6165,
                 hidden_dim = 512,
                 activation = nn.ReLU
                ):
        super().__init__()
        self.mol_emb_size = mol_emb_size
        self.prot_emb_size = prot_emb_size

        self.mol_projector = nn.Sequential(
            nn.Linear(self.mol_emb_size, hidden_dim),
            activation()
        )

        self.prot_projector = nn.Sequential(
            nn.Linear(self.prot_emb_size, hidden_dim),
            activation()
        )
        
        self.fc = nn.Linear(2*hidden_dim, 1)

    def forward(self, mol_emb, prot_emb):
        mol_proj = self.mol_projector(mol_emb)
        prot_proj = self.prot_projector(prot_emb)
        # print(mol_proj.shape, prot_proj.shape)
        cat_emb = torch.cat([mol_proj, prot_proj],axis=1)
        # print(cat_emb.shape)
        return self.fc(cat_emb).squeeze()

In [83]:
model = SimplePLMModel().cuda()

In [84]:
from torch.utils.data import DataLoader
from plm_dti import DTIDataset, molecule_protein_collate_fn

dti_dataset = DTIDataset(
    train_val.Drug,
    train_val.Target,
    train_val.Y,
    mol_featurizer,
    prot_featurizer,
)

dti_dataloader = DataLoader(
    dti_dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=lambda x: molecule_protein_collate_fn(x, pad=False))

In [85]:
from torch.autograd import Variable

n_epo = 25
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
every_n_val = 1
loss_history = []

for epo in tqdm(range(n_epo)):
    model.train()
    for i, (d, p, label) in enumerate(dti_dataloader):
        
        score = model(d.cuda(), p.cuda())
        label = Variable(torch.from_numpy(np.array(label)).float()).cuda()

        loss_fct = torch.nn.MSELoss()

        loss = loss_fct(score, label)
        loss_history.append((epo, i, float(loss.cpu().detach().numpy())))

        opt.zero_grad()
        loss.backward()
        opt.step()

        if (i % 1000 == 0):
            print('Training at Epoch ' + str(epo + 1) + ' iteration ' + str(i) + ' with loss ' + str(
                loss.cpu().detach().numpy()))

  0%|          | 0/25 [00:00<?, ?it/s]

Training at Epoch 1 iteration 0 with loss 38.08734
Training at Epoch 1 iteration 1000 with loss 2.0454583
Training at Epoch 1 iteration 2000 with loss 3.1446316
Training at Epoch 1 iteration 3000 with loss 4.0732503
Training at Epoch 1 iteration 4000 with loss 6.466305
Training at Epoch 1 iteration 5000 with loss 3.009367
Training at Epoch 2 iteration 0 with loss 2.7679374
Training at Epoch 2 iteration 1000 with loss 3.622054
Training at Epoch 2 iteration 2000 with loss 1.7874537
Training at Epoch 2 iteration 3000 with loss 2.739191
Training at Epoch 2 iteration 4000 with loss 1.9332738
Training at Epoch 2 iteration 5000 with loss 1.5109687
Training at Epoch 3 iteration 0 with loss 2.9467134
Training at Epoch 3 iteration 1000 with loss 3.2371593
Training at Epoch 3 iteration 2000 with loss 2.9080515
Training at Epoch 3 iteration 3000 with loss 3.9319758
Training at Epoch 3 iteration 4000 with loss 3.3324797
Training at Epoch 3 iteration 5000 with loss 2.7922795
Training at Epoch 4 iter

In [91]:
running_loss = [float(i[2].cpu().detach().numpy()) for i in loss_history]

In [86]:
test_dataset = DTIDataset(
    test.Drug,
    test.Target,
    test.Y,
    mol_featurizer,
    prot_featurizer,
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=lambda x: molecule_protein_collate_fn(x, pad=False))

In [87]:
pred_list = []

model.eval()
with torch.no_grad():
    for i, (d, p, label) in enumerate(test_dataloader):
        score = model(d.cuda(), p.cuda())
        score = score.detach().cpu().numpy()
        pred_list.extend(score)
    
pred_list = np.array(pred_list)
predictions = {name: pred_list}

In [88]:
out = group.evaluate(predictions)

In [89]:
out

{'bindingdb_patent': {'pcc': 0.504}}